### --- Day 4: Repose Record ---
You've sneaked into another supply closet - this time, it's across from the prototype suit manufacturing lab. You need to sneak inside and fix the issues with the suit, but there's a guard stationed outside the lab, so this is as close as you can safely get.

As you search the closet for anything that might help, you discover that you're not the first person to want to sneak in. Covering the walls, someone has spent an hour starting every midnight for the past few months secretly observing this guard post! They've been writing down the ID of the one guard on duty that night - the Elves seem to have decided that one guard was enough for the overnight shift - as well as when they fall asleep or wake up while at their post (your puzzle input).

For example, consider the following records, which have already been organized into chronological order:
```
[1518-11-01 00:00] Guard #10 begins shift
[1518-11-01 00:05] falls asleep
[1518-11-01 00:25] wakes up
[1518-11-01 00:30] falls asleep
[1518-11-01 00:55] wakes up
[1518-11-01 23:58] Guard #99 begins shift
[1518-11-02 00:40] falls asleep
[1518-11-02 00:50] wakes up
[1518-11-03 00:05] Guard #10 begins shift
[1518-11-03 00:24] falls asleep
[1518-11-03 00:29] wakes up
[1518-11-04 00:02] Guard #99 begins shift
[1518-11-04 00:36] falls asleep
[1518-11-04 00:46] wakes up
[1518-11-05 00:03] Guard #99 begins shift
[1518-11-05 00:45] falls asleep
[1518-11-05 00:55] wakes up
```
Timestamps are written using year-month-day hour:minute format. The guard falling asleep or waking up is always the one whose shift most recently started. Because all asleep/awake times are during the midnight hour (00:00 - 00:59), only the minute portion (00 - 59) is relevant for those events.

Visually, these records show that the guards are asleep at these times:
```
Date   ID   Minute
            000000000011111111112222222222333333333344444444445555555555
            012345678901234567890123456789012345678901234567890123456789
11-01  #10  .....####################.....#########################.....
11-02  #99  ........................................##########..........
11-03  #10  ........................#####...............................
11-04  #99  ....................................##########..............
11-05  #99  .............................................##########.....
```
The columns are Date, which shows the month-day portion of the relevant day; ID, which shows the guard on duty that day; and Minute, which shows the minutes during which the guard was asleep within the midnight hour. (The Minute column's header shows the minute's ten's digit in the first row and the one's digit in the second row.) Awake is shown as ., and asleep is shown as #.

Note that guards count as asleep on the minute they fall asleep, and they count as awake on the minute they wake up. For example, because Guard #10 wakes up at 00:25 on 1518-11-01, minute 25 is marked as awake.

If you can figure out the guard most likely to be asleep at a specific time, you might be able to trick that guard into working tonight so you can have the best chance of sneaking in. You have two strategies for choosing the best guard/minute combination.

Strategy 1: Find the guard that has the most minutes asleep. What minute does that guard spend asleep the most?

In the example above, Guard #10 spent the most minutes asleep, a total of 50 minutes (20+25+5), while Guard #99 only slept for a total of 30 minutes (10+10+10). Guard #10 was asleep most during minute 24 (on two days, whereas any other minute the guard was asleep was only seen on one day).

While this example listed the entries in chronological order, your entries are in the order you found them. You'll need to organize them before they can be analyzed.

**What is the ID of the guard you chose multiplied by the minute you chose?** (In the above example, the answer would be 10 * 24 = 240.)

In [290]:
import pandas as pd
import re
x = []
with open('input.txt', 'r') as f:
    for line in f:
        x.append(line.strip())
res = []
reg = re.compile(r'\[[\d]+-([\d]+-[\d]+)\s(\d{2}:\d{2})\]\s(?:Guard\s#(\d+)\s)?(?:.*(begin|asleep|wakes))')
x = sorted(x)
next_entry = 0
for i in range(len(x)):
    if i != next_entry: continue
    loop = True
    hold = []
    record = x[i]
    if 'Guard' in record:
        hold.append(reg.findall(record)[0])
        while loop:
            i += 1
            if i < len(x):
                t = reg.findall(x[i])[0]
            else:
                loop = False
            if not t[2]:
                hold.append(t)
            else:
                loop = False
                next_entry = i
    else:
        print("ERROR")
    res.append(hold)

res2 = []
for r1 in res:
    d = {x:0 for x in range(60)}
    hold = []
    for i, r2 in enumerate(r1):
        if i == 0:
            try:
                if r2[0] != r1[1][0]:
                    d['date'] = r1[1][0]
                else:
                    d['date'] = r2[0]
            except IndexError:
                d['date'] = r2[0]
            d['guard'] = r2[2]
        for j in range(1, len(r1)):
            hold.append(int(r1[j][1][-2:]))
        break
    hold = list(zip(hold[::2], hold[1::2]))
    for a, w in hold:
        for i in range(a, w):
            d[i] += 1
    res2.append(d)

df = pd.DataFrame(res2)
df['sum'] = df.drop(['guard', 'date'], axis=1).sum(axis=1)
df['sum'] = df.groupby('guard')['sum'].transform(sum)
df = df.sort_values(by='sum', ascending=False).reset_index(drop=True)
guard = df.loc[0, 'guard']
hold = []
for m in range(60):
    hold.append(df[df['guard'].eq(guard)].loc[:, m].sum())
minute = pd.Series(hold).idxmax()
print('Guard#:   {}\nMinute:    {}\nAnswer: {}'.format(guard, minute, int(guard) * minute))

Guard#:   521
Minute:    24
Answer: 12504


### --- Part Two ---
Strategy 2: Of all guards, which guard is most frequently asleep on the same minute?

In the example above, Guard #99 spent minute 45 asleep more than any other guard or minute - three times in total. (In all other cases, any guard spent any minute asleep at most twice.)

What is the ID of the guard you chose multiplied by the minute you chose? (In the above example, the answer would be 99 * 45 = 4455.)

In [303]:
df2 = []
for g in df['guard'].unique():
    hold = []
    for m in range(60):
        hold.append(df[df['guard'].eq(g)].loc[:, m].sum())
    df2.append({'guard': g, 'minute': pd.Series(hold).idxmax(), 'max': pd.Series(hold).max()})
df2 = pd.DataFrame(df2)
df2 = df2.sort_values(by='max', ascending=False).reset_index(drop=True)
guard = int(df2.loc[0, 'guard'])
minute = int(df2.loc[0, 'minute'])
print('Guard#:   {}\nMinute:     {}\nAnswer: {}'.format(guard, minute, guard * minute))

Guard#:   2969
Minute:     47
Answer: 139543


In [304]:
df

,0,1,2,3,4,5,6,7,8,9,...,53,54,55,56,57,58,59,date,guard,sum
0,0,0,0,0,1,1,1,1,1,1,...,1,1,0,0,0,0,0,08-16,521,517
1,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,07-30,521,517
2,0,0,0,0,0,0,0,0,0,0,...,1,1,1,0,0,0,0,03-21,521,517
3,0,0,0,1,1,1,1,1,1,1,...,0,0,0,1,1,1,0,08-15,521,517
4,0,0,0,0,0,0,1,1,1,1,...,0,0,0,0,0,0,0,10-22,521,517
5,0,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,0,0,09-26,521,517
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,03-26,521,517
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,07-06,521,517
8,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,05-20,521,517
9,0,0,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,09-15,521,517
